In [1]:
import sys
import time
import os.path
from glob import glob
from datetime import datetime, timedelta

# data tools
import h5py
import numpy as np
import numba as nb
import pandas as pd
import netCDF4 as nc

# custom tools
sys.path.insert(0, '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/')

from namelist_casper import * 

#### SL indices (same as GEFS compress)

* `IxIy_unique`: a 2-d array of `(indx, indy)`; it extracts grid points that are needed for AnEn-SL

* `SL_xy`: a tuple of 12 arrays; one array per month. Each array has `(2574, 20)` elements. `2574` is the total number of BC grid points; `20` represents a BC grid point and its 19 SLs. Array values are indics of `IxIy_unique`

* NA West grids -- (`IxIy_unique`) --> unique grids for AnEn -- (`SL_xy`) --> SLs, multiple BC grids can have the same SLs. 

In [2]:
# supplemental locations
SL_xy_dict = {}
with h5py.File(save_dir+'SL20_d4_unique.hdf', 'r') as h5io:
    IxIy_unique = h5io['unique_inds'][...]
    for i in range(12):
        temp = h5io['mon_{}_inds'.format(i)][...]
        temp = temp.astype(int)
        SL_xy_dict['{}'.format(i)] = temp
IxIy_unique = IxIy_unique.astype(int)
SL_xy = tuple(SL_xy_dict.values())

In [3]:
year = 2000
lead = 2

with h5py.File(ERA_dir+'ERA5_GEFS-fcst_{}.hdf'.format(year), 'r') as h5io:
    era_temp = h5io['era_fcst'][:, lead, ...][..., IxIy_unique[:, 0], IxIy_unique[:, 1]]

In [4]:
era_temp.shape

(366, 7641)

### Scripts

This notebook is now `En_mean_compress.py`